In [2]:
import graphviz
import ast
from graphviz import Digraph
import inspect

# Define the function you want to visualize
def my_function(a, b):
    if a > b:
        c = a - b
    else:
        c = b - a
    return c

# Parse the function into an abstract syntax tree (AST)
ast_tree = ast.parse(inspect.getsource(my_function))

# Define a graph object
graph = Digraph()


In [3]:
# Traverse the AST to create the graph
def visit_node(node):
    if isinstance(node, ast.If):
        # Add a node for the if statement
        graph.node(str(node.lineno), 'if')
        # Recurse on the test condition
        visit_node(node.test)
        # Recurse on the body of the if statement
        for stmt in node.body:
            visit_node(stmt)
        # Recurse on the body of the else statement (if it exists)
        for stmt in node.orelse:
            visit_node(stmt)
    elif isinstance(node, ast.Compare):
        # Add a node for the comparison operator
        graph.node(str(node.lineno), ast.dump(node.ops[0]))
        # Recurse on the left and right operands of the comparison
        visit_node(node.left)
        for op in node.comparators:
            visit_node(op)
    elif isinstance(node, ast.Name):
        # Add a node for the variable name
        graph.node(str(node.lineno), node.id)
    elif isinstance(node, ast.Num):
        # Add a node for the numeric constant
        graph.node(str(node.lineno), str(node.n))
    elif isinstance(node, ast.BinOp):
        # Add a node for the binary operator
        graph.node(str(node.lineno), ast.dump(node.op))
        # Recurse on the left and right operands of the binary operator
        visit_node(node.left)
        visit_node(node.right)

In [4]:
visit_node(ast_tree)

In [6]:
# Add edges between the nodes
def add_edges(node):
    if isinstance(node, ast.If):
        # Add edges between the if statement and its components
        graph.edge(str(node.lineno), str(node.test.lineno))
        graph.edge(str(node.lineno), str(node.body[0].lineno))
        if len(node.orelse) > 0:
            graph.edge(str(node.lineno), str(node.orelse[0].lineno))
        # Recurse on the components of the if statement
        add_edges(node.test)
        for stmt in node.body:
            add_edges(stmt)
        for stmt in node.orelse:
            add_edges(stmt)
    elif isinstance(node, ast.Compare):
        # Add edges between the comparison operator and its operands
        graph.edge(str(node.lineno), str(node.left.lineno))
        for op in node.comparators:
            graph.edge(str(node.lineno), str(op.lineno))
        # Recurse on the operands of the comparison operator
        add_edges(node.left)
        for op in node.comparators:
            add_edges(op)
    elif isinstance(node, ast.Name):
        pass
    elif isinstance(node, ast.Num):
        pass
    elif isinstance(node, ast.BinOp):
        # Add edges between the binary operator and its operands
        graph.edge(str(node.lineno), str(node.left.lineno))
        graph.edge(str(node.lineno), str(node.right.lineno))
        # Recurse on the operands of the binary operator
        add_edges(node.left)
        add_edges(node.right)

add_edges(ast_tree)

# Render the graph
graph.render("tessss", format="png")



'tessss.png'